In [1]:
import os
import sys
import importlib
from pathlib import Path

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

# %load_ext autoreload
# %autoreload 2

In [2]:
# MAIN DIR
main_dir = f'{str(Path(code_dir).parents[0])}/'

# code_dir
code_dir = f'{code_dir}/'
sys.path.append(code_dir)

# scraping dir
scraped_data = f'{code_dir}scraped_data/'

# data dir
data_dir = f'{code_dir}data/'

# lang models dir
llm_path = f'{data_dir}Language Models'

# sites
site_list=['Indeed', 'Glassdoor', 'LinkedIn']

# columns
cols=['Sector', 
      'Sector Code', 
      'Gender', 
      'Age', 
      'Language', 
      'Dutch Requirement', 
      'English Requirement', 
      'Gender_Female', 
      'Gender_Mixed', 
      'Gender_Male', 
      'Age_Older', 
      'Age_Mixed', 
      'Age_Younger', 
      'Gender_Num', 
      'Age_Num', 
      '% Female', 
      '% Male', 
      '% Older', 
      '% Younger']

int_variable: str = 'Job ID'
str_variable: str = 'Job Description'
gender: str = 'Gender'
age: str = 'Age'
language: str = 'en'
str_cols = ['Search Keyword', 'Platform', 'Job ID', 'Job Title', 'Company Name', 'Location', 'Job Description', 'Company URL', 'Job URL', 'Tracking ID']
pattern = r'[\n]+|[,]{2,}|[|]{2,}|[\n\r]+|(?<=[a-z]\.)(?=\s*[A-Z])|(?=\:+[A-Z])'
pattern_numbers = r'[^a-zA-Z0-9]'

In [3]:
import string
import re
import time
import json
import csv
import glob
import pickle
import pandas as pd
import numpy as np
import pygwalker as pyg
import googletrans
from googletrans import Translator
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tag import pos_tag, pos_tag_sents
from spacy.pipeline import Sentencizer


In [12]:
df_jobs = pd.read_pickle(f'{data_dir}df_jobs_raw_dropped.pkl').reset_index(drop=True)


In [13]:
len(df_jobs)


62577

In [14]:
df_jobs['Job Description'] = df_jobs.loc[df_jobs['Job Description'].notnull(), 'Job Description'].apply(
    lambda job_description: ' '.join(job_description.split('/')) if '/' in job_description else job_description
)


In [15]:
abb_dict = {
    r'incl\.': 'including', 
    r'e\.g\.': 'for example', 
    r'e\.g': 'for example', 
    r'etc\.': 'et cetera', 
}


In [16]:
str_fix_incl = 'Apply appropriate and effective communication methods to senior management and important stakeholders'
str_fix_eg = 'Partner with Procurement in order to manage suppliers for the projects & programs in scope'



In [17]:
df_jobs['Job Description'] = df_jobs['Job Description'].replace(abb_dict, regex=True)

In [ ]:
df_jobs['Job Description'][df_jobs['Job Description'].str.contains(str_fix_incl)]


In [ ]:
re.findall(str_fix_eg, df_jobs['Job Description'][35])

In [ ]:
list(re.finditer(str_fix_eg, df_jobs['Job Description'][35]))

In [ ]:
df_jobs['Job Description'][35][1461:1580]

In [ ]:
# df_jobs['Job Description'] = df_jobs.loc[df_jobs['Job Description'].notnull(), 'Job Description'].apply(
#     lambda job_description: job_description.replace('incl.', 'including')
# )

# df_jobs['Job Description'] = df_jobs.loc[df_jobs['Job Description'].notnull(), 'Job Description'].apply(
#     lambda job_description: job_description.replace('e.g.', 'e.g')
# )

# df_jobs['Job Description'] = df_jobs.loc[df_jobs['Job Description'].notnull(), 'Job Description'].apply(
#     lambda job_description: job_description.replace('e.g', 'for example')
# )

# df_jobs['Job Description'] = df_jobs.loc[df_jobs['Job Description'].notnull(), 'Job Description'].apply(
#     lambda job_description: job_description.replace('etc.', 'et cetera')
# )


In [ ]:
# df_jobs = df_jobs[:100]

In [ ]:
# df_jobs = df_jobs.sample(n=100, random_state=1)

In [ ]:
# df_jobs = pd.read_pickle(f'{data_dir}df_jobs_raw_glob_paths_10.pkl').reset_index(drop=True)


In [ ]:
df_jobs.info()


In [ ]:
dutch_requirement_pattern = r'[Ll]anguage: [Dd]utch|[Dd]utch [Pp]referred|[Dd]utch [Re]quired|[Dd]utch [Ll]anguage|[Pp]roficient in [Dd]utch|[Ss]peak [Dd]utch|[Kk]now [Dd]utch'
english_requirement_pattern = r'[Ll]anguage: [Ee]nglish|[Ee]nglish [Pp]referred|[Ee]nglish [Re]quired|[Ee]nglish [Ll]anguage|[Pp]roficient in [Ee]nglish|[Ss]peak [Ee]nglish|[Kk]now [Ee]nglish'


In [ ]:
# Language requirements
# Dutch
if 'Dutch Requirement' in df_jobs.columns:
    df_jobs.drop(columns=['Dutch Requirement'], inplace=True)
df_jobs['Dutch Requirement'] = np.where(
    df_jobs['Job Description'].str.contains(dutch_requirement_pattern),
    'Yes',
    'No',
)

# English
if 'English Requirement' in df_jobs.columns:
    df_jobs.drop(columns=['English Requirement'], inplace=True)
df_jobs['English Requirement'] = np.where(
    df_jobs['Job Description'].str.contains(english_requirement_pattern),
    'Yes',
    'No',
)

In [ ]:
df_jobs['Dutch Requirement'].value_counts()


In [ ]:
df_sectors = pd.read_pickle(f'{scraped_data}CBS/Data/Sectors Output from script.pkl').reset_index(drop=True)


In [ ]:
df_sectors.columns = [
    '_'.join(col) 
    if 'SBI Sector Titles' not in col 
    and 'Total Workforce' not in col 
    else col[-1] 
    for col in df_sectors.columns
]


In [ ]:
df_sectors.columns


In [ ]:
df_sectors.rename(
    columns={
        'Keywords': 'Search Keyword', 
        'Code': 'Sector Code', 
        'Sector Name': 'Sector', 
        'Gender_Female_n': 'Female Count (x 1000)', 
        'Gender_Male_n': 'Male Count (x 1000)', 
        'Gender_Sectoral Gender Segregation_Dominant Category': 'Gender', 
        'Age_Sectoral Age Segregation_Dominant Category': 'Age', 
        'n': 'Sector Count (x 1000)', 
    },
    inplace=True
)


In [ ]:
type(df_sectors['Search Keyword'][1])

In [ ]:
df_sectors = df_sectors.explode(
    'Search Keyword', ignore_index=True
).reset_index(drop=True)


In [ ]:
df_jobs = df_jobs.merge(df_sectors, on='Search Keyword', how='left')


In [ ]:
len(df_jobs)


In [ ]:
df_jobs.columns


In [ ]:
if df_jobs['Search Keyword'].loc[df_jobs['Sector'].isna().sum()] != 0:
    print('Some search keywords did not match a sector. Fixing')
    print(set(df_jobs['Search Keyword'].loc[df_jobs['Sector'].isna()].to_list()))
    print(len(df_jobs['Search Keyword'].loc[df_jobs['Search Keyword'].isin(list(keyword_trans_dict.keys()))]))
    df_jobs = fix_keywords(df_jobs)
    print(set(df_jobs['Search Keyword'].loc[df_jobs['Sector'].isna()].to_list()))
    print(len(df_jobs['Search Keyword'].loc[df_jobs['Search Keyword'].isin(list(keyword_trans_dict.keys()))]))


In [ ]:
len(df_jobs.loc[(df_jobs['Job Description'].str.contains(str_fix_incl)) | (df_jobs['Job Description'].str.contains(str_fix_eg))])


In [ ]:
# job_descriptions = list(set(df_jobs['Job Description'].to_list()))


In [18]:
job_descriptions = list(
    set(
        df_jobs['Job Description'].loc[
            (df_jobs['Job Description'].str.contains(str_fix_incl)) |
            (df_jobs['Job Description'].str.contains(str_fix_eg))
        ].to_list()
    )
)


In [19]:
len(job_descriptions)


2

In [20]:
job_descriptions[0][1500:2000]

'and important stakeholders throughout the project lifecycle.\nAs conflicts and escalations arise within projects, identify solutions and drive their resolution in a timely and appropriate manner.\nManage change within projects and ensure changes are smoothly and successfully implemented to achieve lasting benefits.\nProject Controlling\nManage project controlling as an independent element to ensure that actual project costs are in line the committed budget.\nConduct engagement reviews. Verify complia'

In [ ]:
df_jobs = df_jobs.loc[
    (df_jobs['Job Description'].str.contains(str_fix_incl)) | 
    (df_jobs['Job Description'].str.contains(str_fix_eg))
].reset_index(drop=True)

In [ ]:
len(df_jobs)

In [ ]:
df_jobs.columns

In [21]:
# Function to make a list of punctuations that determine sentence boundry, i.e., split characters
def make_custom_punct_chars(main_punct_chars = [':', '|'], repeated_punct_chars = ['\n', ',']):
    custom_punct_chars = []
    temp_multi = []
    temp_spaced = []

    for punct_char in main_punct_chars:
        custom_punct_chars+= f'{punct_char}', f'{punct_char} '

    for idx in range(4):
        for punct_char in repeated_punct_chars:
            temp_multi.append(f'{punct_char}'*int(idx+1))
            temp_spaced.append(f'{punct_char} '*int(idx+1))

    for multi, spaced in zip(temp_multi, temp_spaced):
        custom_punct_chars+= multi, spaced

    custom_punct_chars.remove(',')
    custom_punct_chars.remove(', ')

    return custom_punct_chars


In [22]:
custom_punct_chars = make_custom_punct_chars()

In [23]:
custom_punct_chars

[':',
 ': ',
 '|',
 '| ',
 '\n',
 '\n ',
 '\n\n',
 '\n \n ',
 ',,',
 ', , ',
 '\n\n\n',
 '\n \n \n ',
 ',,,',
 ', , , ',
 '\n\n\n\n',
 '\n \n \n \n ',
 ',,,,',
 ', , , , ']

In [24]:
# Set up Spacy
import spacy
from spacy.symbols import NORM, ORTH, LEMMA, POS

nlp = spacy.load('en_core_web_sm')


In [25]:
# Add sentencizer to nlp pip and set custom punctuations
sentencizer = nlp.add_pipe('sentencizer')
sentencizer.punct_chars.update(custom_punct_chars)

# if all(custom_punct_char in sentencizer.punct_chars for custom_punct_char in custom_punct_chars):
#     with open(f'{data_dir}punctuations.txt', 'wb') as f:
#         pickle.dump(sentencizer.punct_chars, f)

# with open(f'{data_dir}punctuations.txt', 'rb') as f:
#     custom_punct_chars = pickle.load(f)


In [ ]:
# if all(custom_punct_char in sentencizer.punct_chars for custom_punct_char in custom_punct_chars):
#     with open(f'{data_dir}punctuations.txt', 'wb') as f:
#         pickle.dump(sentencizer.punct_chars, f)

In [ ]:
# with open(f'{data_dir}punctuations.txt', 'rb') as f:
#     custom_punct_char = pickle.load(f)

In [26]:
from spacy.symbols import NORM, ORTH, LEMMA

special_cases_dict = {
    'incl.': [{65: 'incl', 67: 'including'}],
    'incl. ': [{65: 'incl', 67: 'including'}],
    '(incl.': [{65: 'incl', 67: 'including'}],
    'etc.': [{65: 'etc', 67: 'et cetera'}],
    'etc. ': [{65: 'etc', 67: 'et cetera'}],
    'e.g.': [{65: 'e.g', 67: 'for example'}],
    'e.g. ': [{65: 'e.g', 67: 'for example'}],
}

nlp.tokenizer.rules.update(special_cases_dict)


In [ ]:
nlp.tokenizer.rules['e.g.']

In [ ]:
if all(custom_punct_char in sentencizer.punct_chars for custom_punct_char in custom_punct_chars):
    df_jobs['Job Description spacy_sentencized'] = df_jobs['Job Description'].apply(
        lambda job_description: [
            sent 
            for sentence in nlp(job_description).sents 
            for sent in re.split(pattern, sentence.text) 
            if len(sent) != 0 
        ]
    )


In [ ]:
df_jobs['Job Description spacy_sentencized']


In [ ]:
df_jobs = df_jobs.explode('Job Description spacy_sentencized', ignore_index=True).reset_index(drop=True)


In [ ]:
df_jobs['Job Description spacy_sentencized_lower'] = df_jobs['Job Description spacy_sentencized'].apply(
    lambda job_sentence: job_sentence.strip().lower()
)


In [ ]:
df_jobs.head()

In [ ]:
len(df_jobs)

In [ ]:
# Spacy tokenize
df_jobs['Job Description spacy_tokenized'] = df_jobs['Job Description spacy_sentencized'].apply(
    lambda job_sentence: [
        str(token.text.strip().lower()) 
        for token in nlp.tokenizer(job_sentence) 
        if len(token) != 0 
        and not token.is_stop 
        and not token.is_punct 
        and not token.text in custom_punct_chars
    ]
)


In [ ]:
df_jobs['Job Description spacy_tokenized']

In [27]:
# Load NLK
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer, SnowballStemmer, PorterStemmer, LancasterStemmer

nltk_path = f'{llm_path}/nltk'
nltk.data.path.append(nltk_path)

nltk.download('words', download_dir = nltk_path)
nltk.download('stopwords', download_dir = nltk_path)
nltk.download('punkt', download_dir = nltk_path)
nltk.download('averaged_perceptron_tagger', download_dir = nltk_path)

stop_words = set(stopwords.words('english'))
punctuations = list(string.punctuation)


[nltk_data] Downloading package words to
[nltk_data]     /Users/nyxinsane/Documents/Work - UvA/Automating
[nltk_data]     Equity/Study 1/Study1_Code/data/Language
[nltk_data]     Models/nltk...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nyxinsane/Documents/Work - UvA/Automating
[nltk_data]     Equity/Study 1/Study1_Code/data/Language
[nltk_data]     Models/nltk...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/nyxinsane/Documents/Work - UvA/Automating
[nltk_data]     Equity/Study 1/Study1_Code/data/Language
[nltk_data]     Models/nltk...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/nyxinsane/Documents/Work - UvA/Automating
[nltk_data]     Equity/Study 1/Study1_Code/data/Language
[nltk_data]     Models/nltk...
[nltk_data]   Package averaged_perceptron_

In [28]:
# pattern = r'[\n\r]+|(?<=[a-z]\.)(?=\s*[A-Z])|(?<=[a-z])(?=[A-Z])'
pattern = r'[\n]+|[,]{2,}|[|]{2,}|[\n\r]+|(?<=[a-z]\.)(?=\s*[A-Z])|(?=\:+[A-Z])'

pattern_numbers = r'[^a-zA-Z0-9]'

In [ ]:
# nltk_sentencized = []

# for job_description in job_descriptions:
#     for sentence in sent_tokenize(job_description):
#         for sent in re.split(pattern, sentence):
#             nltk_sentencized.append(sent)

In [29]:
nltk_sentencized = [
    sent 
    for job_description in job_descriptions 
    for sentence in sent_tokenize(job_description) 
    for sent in re.split(pattern, sentence)
    if len(sent) != 0
]

In [30]:
len(nltk_sentencized)

133

In [ ]:
# df_jobs['Job Description nltk_sentencized'] = df_jobs['Job Description'].apply(
#     lambda job_description: [
#         sent 
#         for sentence in sent_tokenize(job_description) 
#         for sent in re.split(pattern, sentence)
#         if len(sent) != 0
#     ]
# )

In [ ]:
# df_jobs['Job Description nltk_sentencized'] = df_jobs['Job Description'].apply(
#     lambda job_description: [
#         re.split(pattern, sentence)
#         for sentence in sent_tokenize(job_description)
#         if len(sentence) != 0
#     ]
# )

In [ ]:
# df_jobs['Job Description nltk_sentencized'][0][-10:-1]

In [ ]:
# len(df_jobs['Job Description nltk_sentencized'][0]) + len(df_jobs['Job Description nltk_sentencized'][1])


In [ ]:
# df_jobs = df_jobs.explode('Job Description nltk_sentencized', ignore_index=True).reset_index(drop=True)

In [ ]:
# len(df_jobs)

In [ ]:
# df_jobs['Job Description nltk_sentencized']

In [ ]:
nltk_sentencized_lower = [
    str(sent.strip().lower()) 
    for sent in nltk_sentencized
]

In [ ]:
len(nltk_sentencized_lower)

In [ ]:
# df_jobs['Job Description nltk_sentencized_lower'] = df_jobs['Job Description nltk_sentencized'].apply(
#     lambda job_sentence: job_sentence.strip().lower()
# )


In [ ]:
# df_jobs['Job Description nltk_sentencized_lower']


In [ ]:
for index, sentence in df_jobs['Job Description spacy_sentencized'].items():
    if str_fix_eg in sentence:
        print(index)


In [ ]:
df_jobs['Job Description spacy_sentencized'][17:25].to_list()

In [ ]:
for index, sentence in df_jobs['Job Description spacy_sentencized'].items():
    if 'Power' in sentence:
        print(index)

In [ ]:
df_jobs['Job Description spacy_sentencized'][60:70].to_list()

In [ ]:
for idx, sent in enumerate(nltk_sentencized):
    if str_fix_eg in sent:
        print(idx)

In [ ]:
next(idx for idx, sent in enumerate(nltk_sentencized) if str_fix_eg in sent)

In [ ]:
# str_fix = 'Apply appropriate and effective communication methods to senior management and important stakeholders \(incl.'

for idx, sent in enumerate(nltk_sentencized):
    if str_fix_incl.split('\(')[0] in sent:
        print(idx)

In [ ]:
nltk_sentencized[84:95]

In [ ]:
# sentences split on e.g. and incl.
nltk_sentencized[
    next(idx for idx, sent in enumerate(nltk_sentencized) if str_fix_eg in sent):next(idx for idx, sent in enumerate(nltk_sentencized) if str_fix_incl.split('\(')[0] in sent)+2
]


In [ ]:
for idx, sent in enumerate(nltk_sentencized):
    if 'Power' in sent:
        print(idx)

In [ ]:
next(idx for idx, sent in enumerate(nltk_sentencized) if 'Power' in sent)

In [ ]:
nltk_sentencized[next(idx for idx, sent in enumerate(nltk_sentencized) if 'Power' in sent)]

In [ ]:
for idx, sentence in enumerate(nltk_sentencized[84:95]):
    print(f'Sentence {idx+1}: {sentence}\n')

In [ ]:
# nltk_tokenized = []

# for job_sentence in nltk_sentencized:
#     for token in word_tokenize(job_sentence):
#         if len(token) != 0 and token != '...' and token.lower() not in set(stopwords.words('english')) and token.lower() not in list(string.punctuation):
#             nltk_tokenized.append(str(token.strip().lower())))


In [ ]:
nltk_tokenized = [
    str(token.strip().lower()) 
    for job_sentence in nltk_sentencized 
    for token in word_tokenize(job_sentence) 
    if len(token) != 0 
    and token != '...' 
    and not token.lower() in set(stopwords.words('english')) 
    and not token.lower() in list(string.punctuation) 
]

In [ ]:
df_jobs['Job Description nltk_tokenized'] = df_jobs['Job Description spacy_sentencized'].apply(
    lambda job_sentence: [
        str(token.strip().lower()) 
        for token in word_tokenize(job_sentence) 
        if len(token) != 0 
        and token != '...' 
        and not token.lower() in set(stopwords.words('english')) 
        and not token.lower() in list(string.punctuation) 
    ]
)


In [ ]:
df_jobs['Job Description nltk_tokenized']

In [ ]:
'...' in nltk_tokenized

In [ ]:
len(nltk_tokenized)

In [ ]:
from gensim.utils import save_as_line_sentence, simple_preprocess
from gensim.parsing.preprocessing import remove_stopwords, preprocess_string, preprocess_documents

gensim_tokenized = []

for job_sentence in nltk_sentencized:
    gensim_tokenized.append(preprocess_string(re.sub(pattern_numbers, ' ', job_sentence.strip().lower())))

In [ ]:
gensim_tokenized[0]

In [ ]:
df_jobs['Job Description gensim_tokenized'] = df_jobs['Job Description spacy_sentencized'].apply(
    lambda job_sentence: preprocess_string(re.sub(pattern_numbers, ' ', job_sentence.strip().lower()))
)


In [ ]:
df_jobs['Job Description gensim_tokenized']

In [ ]:
nltk_token_tags = pos_tag(nltk_tokenized)

In [ ]:
nltk_token_tags[1][1]

In [ ]:
df_jobs['Job Description nltk_token_tags'] = df_jobs['Job Description nltk_tokenized'].apply(
    lambda token: pos_tag(token)
)


In [ ]:
df_jobs['Job Description nltk_token_tags']

In [ ]:
# https://www.machinelearningplus.com/nlp/lemmatization-examples-python/

In [ ]:
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([token])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)


In [ ]:
import unicodedata

lemmatizer = WordNetLemmatizer()
stemmer = SnowballStemmer('english')

nltk_lemmas = []
nltk_stems = []

for token in nltk_tokenized:
    token = unicodedata.normalize('NFKD', str(token.strip().lower())).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    token_lemma = lemmatizer.lemmatize(token)
    token_stem = stemmer.stem(token)
    token_pos = get_wordnet_pos(token)
    token_pos_lemma = lemmatizer.lemmatize(token, token_pos)
    print(f'Token: {token} | Stem: {token_stem} | Lemma: {token_lemma} | POS Lemma: {token_pos_lemma}')
    nltk_lemmas.append(token_pos_lemma)
    nltk_stems.append(token_stem)


In [ ]:
import unicodedata

lemmatizer = WordNetLemmatizer()
stemmer = SnowballStemmer('english')

nltk_lemmas = []
nltk_stems = []

for token in nltk_tokenized:
    token = unicodedata.normalize('NFKD', str(token.strip().lower())).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    token_lemma = lemmatizer.lemmatize(token)
    token_stem = stemmer.stem(token)
    token_pos = get_wordnet_pos(token)
    nltk_lemmas.append(lemmatizer.lemmatize(token, get_wordnet_pos(token)))
    nltk_stems.append(token_stem)


In [ ]:
nltk_lemmas = [
    lemmatizer.lemmatize(
        token, get_wordnet_pos(
            unicodedata.normalize('NFKD', str(token.strip().lower())).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        )
    )
    for token in nltk_tokenized
]

In [ ]:
df_jobs['Job Description nltk_lemmas'] = df_jobs['Job Description spacy_tokenized'].apply(
    lambda tokens: [
        lemmatizer.lemmatize(
            token, get_wordnet_pos(
                unicodedata.normalize('NFKD', str(token.strip().lower())).encode('ascii', 'ignore').decode('utf-8', 'ignore')
            )
        )
        for token in tokens
    ]
)


In [ ]:
df_jobs['Job Description nltk_lemmas']

In [ ]:
len(nltk_lemmas)

In [ ]:
import unicodedata

lemmatizer = WordNetLemmatizer()
stemmer = SnowballStemmer('english')

nltk_lemmas = []
nltk_stems = []

for token in nltk_tokenized:
    token = unicodedata.normalize('NFKD', str(token.strip().lower())).encode('ascii', 'ignore').decode('utf-8', 'ignore')

    nltk_stems.append(stemmer.stem(unicodedata.normalize('NFKD', str(token.strip().lower())).encode('ascii', 'ignore').decode('utf-8', 'ignore')))


In [ ]:
nltk_stems = [
    stemmer.stem(
        unicodedata.normalize('NFKD', str(token.strip().lower())).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    )
    for token in nltk_tokenized
]

In [ ]:
df_jobs['Job Description nltk_stems'] = df_jobs['Job Description spacy_tokenized'].apply(
    lambda tokens: [
        stemmer.stem(
            unicodedata.normalize('NFKD', str(token.strip().lower())).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        )
        for token in tokens
    ]
)


In [ ]:
df_jobs['Job Description nltk_stems']

In [ ]:
len(nltk_stems)

In [ ]:
df_jobs.columns

In [ ]:
# special_cases = {":)": [{"ORTH": ":)"}]}
# prefix_re = re.compile(r'''^[\\[\\("']''')
# suffix_re = re.compile(r'''[\\]\\)"']$''')
# infix_re = re.compile(r'''[-~]''')
# simple_url_re = re.compile(r'''^https?://''')

# def custom_tokenizer(nlp):
#     return Tokenizer(nlp.vocab, rules=special_cases,
#                                 prefix_search=prefix_re.search,
#                                 suffix_search=suffix_re.search,
#                                 infix_finditer=infix_re.finditer,
#                                 url_match=simple_url_re.match)

In [ ]:
# nlp.tokenizer.add_special_case('incl', [{ORTH: u'incl', NORM: u'include', LEMMA: u'include', POS: u'VERB'}])


In [ ]:
# # Spacy sentencizer
# spacy_sentencized = []

# if all(custom_punct_char in sentencizer.punct_chars for custom_punct_char in custom_punct_chars):
#     for job_description in job_descriptions:
#         for sentence in nlp(job_description).sents:
#             for sent in re.split(pattern, sentence.text):
#                 if len(sent) != 0:
#                     spacy_sentencized.append(sent)

In [31]:
# Spacy sentencizer
if all(custom_punct_char in sentencizer.punct_chars for custom_punct_char in custom_punct_chars):
    spacy_sentencized = [
        sent 
        for job_description in job_descriptions 
        for sentence in nlp(job_description).sents 
        for sent in re.split(pattern, sentence.text) 
        if len(sent) != 0 
    ]
    
    spacy_sentencized_lower = [
        str(sent.strip().lower()) 
        for sent in spacy_sentencized
    ]


In [ ]:
# # Spacy sentencizer
# if all(custom_punct_char in sentencizer.punct_chars for custom_punct_char in custom_punct_chars):
#     spacy_sentencized = [
#         re.split(pattern, sentence.text) 
#         for job_description in job_descriptions 
#         for sentence in nlp(job_description).sents 
# #         for sent in re.split(pattern, sentence.text) 
#         if len(sentence) != 0 
#     ]



In [32]:
spacy_sentencized[23]

'Conduct engagement reviews.'

In [45]:
# Spacy tokenizer
spacy_tokenized = []

if all(custom_punct_char in sentencizer.punct_chars for custom_punct_char in custom_punct_chars):
    for job_sentence in spacy_sentencized:
        for token in nlp.tokenizer(job_sentence):
            if len(token) != 0 and not token.is_stop and not token.is_punct and token.text not in custom_punct_chars:
                spacy_tokenized.append(token.text)

In [46]:
spacy_tokenized

['Purpose',
 'Manager',
 'Digital',
 'Strategy',
 'Programs',
 'adidas',
 'market',
 'ecom',
 'organization',
 'task',
 'manage',
 'deliver',
 'small',
 'digital',
 'projects',
 'significant',
 'RFCs',
 'sub',
 'elements',
 'mid',
 'scale',
 'digital',
 'projects',
 'project',
 'scope',
 'local',
 'market',
 'level',
 'brands',
 'functions',
 'objective',
 'conduct',
 'projects',
 'boundaries',
 'time',
 'cost',
 'quality',
 'focusing',
 'consumers',
 'expectations',
 'requirements',
 'Planning',
 'executing',
 'tracking',
 'projects',
 'cooperation',
 'respective',
 'functions',
 'project',
 'team',
 'members',
 'examples',
 'expected',
 'role',
 'Additionally',
 'focusing',
 'resource',
 'management',
 'coordination',
 'activities',
 'acting',
 'central',
 'interface',
 'local',
 'stakeholders',
 'Key',
 'Responsibilities',
 'Scope',
 'Manage',
 'deliver',
 'small',
 'digital',
 'projects',
 'local',
 'market',
 'level',
 'Project',
 ' ',
 'Program',
 'Delivery',
 'Deliver',
 'projec

In [ ]:
type(spacy_sentencized[0])

In [ ]:
len(spacy_sentencized)

In [ ]:
for idx, sent in enumerate(spacy_sentencized):
    if str_fix_eg in sent:
        print(idx)

In [ ]:
next(idx for idx, sent in enumerate(spacy_sentencized) if str_fix_eg in sent)

In [ ]:
# str_fix = 'Apply appropriate and effective communication methods to senior management and important stakeholders \(incl.'


for idx, sent in enumerate(spacy_sentencized):
    if str_fix_incl.split('\(')[0] in sent:
        print(idx)

In [ ]:
next(idx for idx, sent in enumerate(spacy_sentencized) if str_fix_incl.split('\(')[0] in sent)

In [ ]:
spacy_sentencized[18:25]

In [ ]:
for idx, sent in enumerate(spacy_sentencized):
    if 'Power' in sent:
        print(idx)

In [ ]:
next(idx for idx, sent in enumerate(spacy_sentencized) if 'Power' in sent)

In [ ]:
spacy_sentencized[63]

In [ ]:
for idx, sentence in enumerate(spacy_sentencized[18:25]):
    print(f'Sentence {idx+1}: {sentence}\n')

In [ ]:
# # Spacy tokenizer
# spacy_tokenized = []

# for job_sentence in spacy_sentencized:
# #         doc = nlp.tokenizer(job_sentence)
#     spacy_tokenized.extend(
#         [
#             token.text for token in nlp.tokenizer(job_sentence) 
#             if token.text not in custom_punct_chars
#             and not token.is_stop 

#         ]
#     )

In [ ]:
type(spacy_sentencized[0])

In [ ]:
spacy_sentencized[6]

In [ ]:
# Spacy tokenizer

spacy_tokenized = [
    str(token.text.strip().lower()) 
    for job_sentence in spacy_sentencized 
    for token in nlp.tokenizer(job_sentence) 
    if len(token) != 0 
    and not token.is_stop 
    and not token.is_punct 
    and not token.text in custom_punct_chars
]


In [ ]:
len(spacy_tokenized)

In [ ]:
spacy_tokenized[:10]

In [ ]:
# nlp.get_pipe('attribute_ruler').add([[{"TEXT":"Angeltown"}]],{"LEMMA":"San Fransisco"})

In [ ]:
# # Spacy pos tagging
# nlp_stemmer = LancasterStemmer()
nlp_stemmer = PorterStemmer()

nlp_token_tags = []
nlp_lemmas = []
nlp_stems = []

for job_description in job_descriptions:
    for token in nlp(job_description):
        if len(token) != 0 and not token.is_stop and not token.is_punct and token.text not in custom_punct_chars:
            nlp_token_tags.append(tuple([token.text, token.tag_]))
            nlp_lemmas.append(token.lemma_)
            nlp_stems.append(nlp_stemmer.stem(token.text))
#         for sentence in doc.sents:
#             for sent in re.split(pattern, sentence.text):
#                 if len(sent) != 0:
#                     spacy_sentencized.append(sent)

In [ ]:
nlp_token_tags = [
    tuple([token.text.strip().lower(), token.tag_])
    for job_description in job_descriptions
    for token in nlp(job_description)
    if len(token) != 0 and not token.is_stop and not token.is_punct and token.text not in custom_punct_chars
]


In [ ]:
df_jobs['Job Description nlp_token_tags'] = df_jobs['Job Description spacy_sentencized'].apply(
    lambda job_sentence: [
        tuple([token.text.strip().lower(), token.tag_])
        for token in nlp(job_sentence)
        
    ]
)


In [ ]:
nlp_lemmas = [
    token.lemma_.strip().lower()
    for job_description in job_descriptions
    for token in nlp(job_description)
    if len(token) != 0 and not token.is_stop and not token.is_punct and token.text not in custom_punct_chars
]

In [ ]:
df_jobs['Job Description nlp_lemmas'] = df_jobs['Job Description spacy_sentencized'].apply(
    lambda job_sentence: [
        token.lemma_.strip().lower()
        for token in nlp(job_sentence)
        if len(token) != 0 and not token.is_stop and not token.is_punct and token.text not in custom_punct_chars
    ]
)


In [ ]:
nlp_stems = [
    stemmer.stem(token.text.strip().lower())
    for job_description in job_descriptions
    for token in nlp(job_description)
    if len(token) != 0 and not token.is_stop and not token.is_punct and token.text not in custom_punct_chars
]

In [ ]:
df_jobs['Job Description nlp_stems'] = df_jobs['Job Description spacy_sentencized'].apply(
    lambda job_sentence: [
        stemmer.stem(token.text.strip().lower())
        for token in nlp(job_sentence)
        if len(token) != 0 and not token.is_stop and not token.is_punct and token.text not in custom_punct_chars
    ]
)


In [ ]:
df_jobs['Job Description nlp_stems']

In [ ]:
# if all(custom_punct_char in sentencizer.punct_chars for custom_punct_char in custom_punct_chars):
#     nlp_token_tags = [
#         tuple([token.text, token.tag_]) 
#         for job_description in job_descriptions 
#         for token in nlp(job_description) 
#         if len(token) != 0 
#         and not token.is_stop 
#         and not token.is_punct 
#         and not token.text in custom_punct_chars
#     ]

In [ ]:
def get_word_num_and_frequency(row, text_col):

    row['Job Description num_words'] = len(str(row[f'{text_col}']).split())
    row['Job Description num_unique_words'] = len(set(str(row[f'{text_col}']).split()))
    row['Job Description num_chars'] = len(str(row[f'{text_col}']))
    row['Job Description num_punctuations'] = len([c for c in str(row[f'{text_col}']) if c in string.punctuation])

    return row


In [ ]:
df_jobs = df_jobs.apply(lambda row: get_word_num_and_frequency(row=row, text_col='Job Description spacy_sentencized'), axis='columns')

In [ ]:
# for sent in spacy_sentencized:
#     num_words = len(sent.split())
#     num_unique_words = len(set(sent.split()))
#     num_chars = len(sent)
#     num_punctuations = len([c for c in sent if c in string.punctuation])

In [ ]:
# Sentiment
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentim_analyzer = SentimentIntensityAnalyzer()

def get_sentiment(df_jobs_to_be_processed, text_col, algo='vader', sentiment_range=(-1,1)):

    ## calculate sentiment
    if algo == 'vader':
        df_jobs_to_be_processed['Sentiment'] = df_jobs_to_be_processed[text_col].apply(lambda x: sentim_analyzer.polarity_scores(x)['compound'] if isinstance(x, str) else np.nan)
    elif algo == 'textblob':
        df_jobs_to_be_processed['Sentiment'] = df_jobs_to_be_processed[text_col].apply(lambda x: TextBlob(x).sentiment.polarity)
    ## rescaled
    if sentiment_range != (-1,1):
        df_jobs_to_be_processed['Sentiment'] = preprocessing.MinMaxScaler(feature_range=sentiment_range).fit_transform(df_jobs_to_be_processed[['sentiment']])
    # print(df_jobs_to_be_processed[['sentiment']].describe().T)

    return df_jobs_to_be_processed


In [ ]:
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob

nlp.add_pipe('spacytextblob')

print()

In [42]:
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob

# nlp.add_pipe('spacytextblob')
sentiment = []
for job_sentence in spacy_sentencized:
    sentiment.append(nlp(job_sentence)._.blob.polarity)

Purpose:
Sentiment(polarity=0.0, subjectivity=0.0, assessments=[])
As a Manager Digital Strategy and Programs in adidas market ecom organization your task is to manage and deliver small digital projects, significant RFCs or sub-elements of mid-scale digital projects.
Sentiment(polarity=0.025, subjectivity=0.255, assessments=[(['digital'], 0.0, 0.0, None), (['small'], -0.25, 0.4, None), (['digital'], 0.0, 0.0, None), (['significant'], 0.375, 0.875, None), (['digital'], 0.0, 0.0, None)])
The project scope is on a local (market) level across brands and functions.
Sentiment(polarity=0.0, subjectivity=0.0, assessments=[(['local'], 0.0, 0.0, None)])
Your objective is to conduct the projects within the boundaries of time, cost & quality while focusing on consumers' expectations and requirements.
Sentiment(polarity=0.0, subjectivity=0.1, assessments=[(['objective'], 0.0, 0.1, None)])
Planning, executing and tracking projects in cooperation with the respective functions and project team members

Sentiment(polarity=0.0, subjectivity=0.1125, assessments=[(['independent'], 0.0, 0.125, None), (['actual'], 0.0, 0.1, None)])
Lead or conduct engagement reviews.
Sentiment(polarity=0.0, subjectivity=0.0, assessments=[])
Verify implementation of quality assurance procedures.
Sentiment(polarity=0.0, subjectivity=0.0, assessments=[])
Ensure project & program delivery against program KPIs.
Sentiment(polarity=0.0, subjectivity=0.0, assessments=[])
Monitor project & program variables (cost, effort, scope, et cetera) against plan in order to implement corrective or preventive actions.
Sentiment(polarity=0.0, subjectivity=0.0, assessments=[])
People Management
Sentiment(polarity=0.0, subjectivity=0.0, assessments=[])
Ensure appropriate leadership skills are present at every level by creating a motivational and supportive work environment in which employees are coached, trained and provided with career opportunities through development.
Sentiment(polarity=0.3333333333333333, subjectivity=0.5, a

In [ ]:
sentiment = [
    float(nlp(job_sentence)._.blob.polarity)
    for job_sentence in spacy_sentencized
]

In [ ]:
df_jobs['Job Description spacy_sentiment'] = df_jobs['Job Description spacy_sentencized'].apply(
    lambda sentence: float(nlp(sentence)._.blob.polarity)
)


In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentim_analyzer = SentimentIntensityAnalyzer()

df_jobs['Job Description nltk_sentiment'] = df_jobs['Job Description spacy_sentencized'].apply(
    lambda sentence: float(sentim_analyzer.polarity_scores(sentence)['compound'])
    if isinstance(sentence, str) else np.nan
)


In [ ]:
df_jobs['Job Description nltk_sentiment']

In [ ]:
df_jobs = get_sentiment(df_jobs, text_col='Job Description spacy_sentencized')


In [ ]:
df_jobs['Sentiment']

In [ ]:
df_jobs.head()

In [ ]:
# Sentiment
from nltk.sentiment.vader import SentimentIntensityAnalyzer

vader = SentimentIntensityAnalyzer()

for sent in spacy_sentencized:
    print(vader.polarity_scores(sent)['compound'])


In [ ]:
# # Sentiment
# from textblob import TextBlob, Word

# for sent in spacy_sentencized:
#     print(TextBlob(sent).sentiment.polarity)


In [ ]:
# Bert Tokenizer
from transformers import BertTokenizer, AutoTokenizer, BertForPreTraining, BertConfig, BertModel

bert_model_name = 'bert-base-uncased'
bert_tokenizer = BertTokenizer.from_pretrained(bert_model_name, strip_accents = True)
bert_model = BertModel.from_pretrained(bert_model_name)

df_jobs['bert_tokenized'] = df_jobs['Job Description spacy_sentencized'].apply(lambda sentence: bert_tokenizer.tokenize(str(sentence)))



In [ ]:
df_jobs['bert_tokenized']

In [ ]:
from gensim.models.phrases import ENGLISH_CONNECTOR_WORDS, Phraser, Phrases

# Bigrams
bigram = Phraser(Phrases(df_jobs['Job Description spacy_tokenized'], connector_words=ENGLISH_CONNECTOR_WORDS, min_count=1, threshold=1))
df_jobs['Job Description gensim_2garms'] = bigram[df_jobs['Job Description spacy_tokenized']]

# Trigrams
trigram = Phraser(Phrases(bigram_sentences, connector_words=ENGLISH_CONNECTOR_WORDS, min_count=1, threshold=1))
df_jobs['Job Description gensim_3garms'] = trigram[bigram_sentences]

In [ ]:
df_jobs['Job Description gensim_3garms']

In [ ]:
for idx, row in df_jobs['Job Description gensim_3garms'].items():
    for token in row:
        if '_' in token:
            if re.search('_([^_]+)_', token):
                print(token)

In [53]:
spacy_sentences = ''
for token in spacy_tokenized:
    spacy_sentences+= token + ' '

In [58]:
spacy_sentences = [spacy_sentences.strip().lower()]

In [76]:
spacy_bigrams = []

for job_sentence in spacy_sentences:
#     print(job_sentence)
    for noun_phrase in list(nlp(job_sentence).noun_chunks):
        print(noun_phrase.merge_noun_chunks)
#         print(noun_phrase.merge(noun_phrase.root.tag_, noun_phrase.root.lemma_, noun_phrase.root.ent_type_))
#         if len(token) != 0 and not token.is_stop and not token.is_punct and token.text not in custom_punct_chars:
#             spacy_tokenized.append(token.text)

AttributeError: 'spacy.tokens.span.Span' object has no attribute 'merge_noun_chunks'

In [ ]:
def huggingface_similarity(sentences, bert_model):

    embeddings = bert_model.encode(sentences, show_progress_bar=True)
    cos_sim = util.cos_sim(embeddings, embeddings)

    #Sort list by the highest cosine similarity score
    all_sentence_combinations = sorted([[cos_sim[i][j], i, j] for i in range(len(cos_sim)-1) for j in range(i+1, len(cos_sim))], key=lambda x: x[0], reverse=True)

    if args['print_enabled'] is True:
        print('Top-5 most similar pairs:')
        for score, i, j in all_sentence_combinations[:5]:
            print(f'{sentences[i]} \t {sentences[j]} \t {cos_sim[i][j]:.4f}')

    if args['save_enabled'] is True:
        bert_model.to_json_file('bert_config.json')

    return all_sentence_combinations, embeddings

In [ ]:
for sentences in spacy_sentencized:
    print(bert_tokenizer.encode_plus(sentences))

In [ ]:
from transformers import AutoTokenizer, BertModel
import torch

tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
model = BertModel.from_pretrained(bert_model_name)

inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
outputs = model(**inputs)

last_hidden_states = outputs.last_hidden_state

In [ ]:
last_hidden_states

In [ ]:
cls(
    BertModel.from_pretrained(
        bert_model_name,
        output_attentions=True,
        output_hidden_states=True,
        output_additional_info=True,
    ),
    BertAligner.from_pretrained(bert_model_name),
)

In [ ]:
# Spacy sentencizer
# spacy_sentencized = []

if all(custom_punct_char in sentencizer.punct_chars for custom_punct_char in custom_punct_chars):
    for job_description in job_descriptions:
        for token in nlp(job_description):
            print(token.text, token.tag_)
#             for sent in re.split(pattern, sentence.text):
#                 if len(sent) != 0:
#                     spacy_sentencized.append(sent)

In [77]:
from spacy.matcher import Matcher
nlp = spacy.load('en_core_web_sm')

matcher = Matcher(nlp.vocab)

bigram_patterns = [
    ['VERB', 'ADJ', 'NOUN'],
    ['NOUN', 'VERB', 'ADV'],
    ['NOUN', 'ADP', 'NOUN'],
    # more rules here...
]

rules = [
    ['VERB', 'ADJ', 'NOUN'],
    ['NOUN', 'VERB', 'ADV'],
    ['NOUN', 'ADP', 'NOUN'],
    # more rules here...
]

trigram_patterns = [[{"POS": i} for i in j] for j in rules]


In [78]:
trigram_patterns

[[{'POS': 'VERB'}, {'POS': 'ADJ'}, {'POS': 'NOUN'}],
 [{'POS': 'NOUN'}, {'POS': 'VERB'}, {'POS': 'ADV'}],
 [{'POS': 'NOUN'}, {'POS': 'ADP'}, {'POS': 'NOUN'}]]

In [ ]:
trigram_patterns = {
    'noun_verb': [{'POS': 'NOUN'}, {'POS': 'VERB'}],
    'verb_noun': [{'POS': 'VERB'}, {'POS': 'NOUN'}],
    'adj_noun': [{'POS': 'ADJ'}, {'POS': 'NOUN'}],
    'adj_propn': [{'POS': 'ADJ'}, {'POS': 'PROPN'}],
}

In [ ]:
for pattern_name, pattern in patterns.items():
    matcher.add(pattern_name, [pattern])


In [ ]:
for job_description in job_descriptions:
    doc = nlp(job_description)
    matches = matcher(doc)

In [84]:
for job_description in job_descriptions:
    spacy_bigrams = [
        nlp(job_description)[start:end].text
        for _, start, end in matcher(nlp(job_description))

    ]

/var/folders/46/q15p556n1dd63z6gkwyh896c0000gn/T/ipykernel_12428/2603420453.py:4: UserWarning: [W036] The component 'matcher' does not have any patterns defined.
  for _, start, end in matcher(nlp(job_description))


In [ ]:
for match_id, start, end in matches:
    
    # Get string representation
    string_id = nlp.vocab.strings[match_id]

    # The matched span
    span = doc[start:end]
    
    print(repr(span.text))
    print(match_id, string_id, start, end)
    print()

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')


In [ ]:
# for sentence in spacy_sentencized:
#     print(model.encode(sentence))



In [ ]:
from transformers.models.electra.modeling_tf_electra import TFElectraMainLayer

In [ ]:
from transformers import AutoTokenizer, AutoModelForPreTraining

tokenizer = AutoTokenizer.from_pretrained("pavanchhatpar/electra-base-sentence-splitter")

model = AutoModelForPreTraining.from_pretrained("pavanchhatpar/electra-base-sentence-splitter", from_tf=True)


In [ ]:
nlp = pipeline('sentence-splitter', model)